# Natural Language Processing with Disaster Tweets  
## Weekly Mini-Project – Kaggle Competition

**GitHub Repository:** <URL to your repo>  
**Kaggle Competition:** https://www.kaggle.com/competitions/nlp-getting-started  

---

# 1. Problem Description and Data Overview

This mini-project participates in the Kaggle challenge “Natural Language Processing with Disaster Tweets.”
The goal is to classify tweets into:

- `1` → the tweet refers to a real disaster
- `0` → the tweet does not refer to a real disaster (jokes, metaphors, news headlines, etc.)

This is a standard supervised NLP text classification task.

### What is NLP?

Natural Language Processing (NLP) involves techniques that allow computers to:

- interpret text,
- extract semantic meaning,
- analyze patterns,
- and make predictions.

In this project, raw text must be transformed into numerical vectors (e.g., TF-IDF, word embeddings) so that a machine learning model can process it.

### Data structure

`train.csv` contains the labeled data:

| Column     | Description |
|------------|-------------|
| id         | Tweet ID |
| keyword    | Optional keyword (frequently missing) |
| location   | Noisy text, often missing |
| text       | The tweet content |
| target     | 1 = disaster, 0 = not disaster |

`test.csv` contains the same structure without the `target` column.

---
